# Practical example. Audiobooks

## Create the machine learning algorithm, HParams Tuning

### Import the relevant libraries

In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

### Data

In [2]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model
Outline, optimizers, loss, early stopping and training

In [3]:

max_epochs = 20
batch_size = 64

input_size = 10
output_size = 2

In [4]:

HP_ACTIVATION = hp.HParam('activation_function', hp.Discrete(['sigmoid', 'relu']))
HP_HIDDEN_LAYER_SIZE = hp.HParam('hidden_layer_size', hp.Discrete([32, 64, 96, 128]))

with tf.summary.create_file_writer(r'Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams = [HP_ACTIVATION, HP_HIDDEN_LAYER_SIZE],
        metrics = [hp.Metric('accuracy', display_name = 'Accuracy')]
    )


In [5]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hparams[HP_HIDDEN_LAYER_SIZE], hparams[HP_ACTIVATION]),
        tf.keras.layers.Dense(hparams[HP_HIDDEN_LAYER_SIZE], hparams[HP_ACTIVATION]),
        tf.keras.layers.Dense(output_size, activation='softmax')
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    
    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    
    # Defining the logging directory
    log_dir = r'Logs/Model 1/fit/' + 'run-{}'.format(session_num)

    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')     
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(validation_inputs)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(validation_targets, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["won't buy", "will buy"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        train_inputs,
        train_targets,
        epochs = max_epochs,
        batch_size = batch_size,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (validation_inputs, validation_targets),
        verbose = 2
    )
    
    
    # Evaluating the model's performance on the validation set
    _, accuracy = model.evaluate(validation_inputs, validation_targets)
    
    # Saving the current model for future reference
    model.save(r'saved_models\Model 1\Run-{}'.format(session_num))
    
    return accuracy


In [6]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar('accuracy', accuracy, step=1)


In [7]:
session_num = 1

for activation in HP_ACTIVATION.domain.values:
    for hidden_layer_size in HP_HIDDEN_LAYER_SIZE.domain.values:
        
        hparams = {
            HP_ACTIVATION: activation,
            HP_HIDDEN_LAYER_SIZE: hidden_layer_size
        }

        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/' + run_name, hparams, session_num)

        session_num += 1


--- Starting trial: run-1
{'activation_function': 'relu', 'hidden_layer_size': 32}
Epoch 1/20
56/56 - 1s - loss: 0.5654 - accuracy: 0.7603 - val_loss: 0.4294 - val_accuracy: 0.8792
Epoch 2/20
56/56 - 1s - loss: 0.3639 - accuracy: 0.8737 - val_loss: 0.3096 - val_accuracy: 0.8904
Epoch 3/20
56/56 - 1s - loss: 0.3122 - accuracy: 0.8821 - val_loss: 0.2878 - val_accuracy: 0.8971
Epoch 4/20
56/56 - 1s - loss: 0.2927 - accuracy: 0.8905 - val_loss: 0.2674 - val_accuracy: 0.8993
Epoch 5/20
56/56 - 0s - loss: 0.2798 - accuracy: 0.8952 - val_loss: 0.2599 - val_accuracy: 0.9016
Epoch 6/20
56/56 - 1s - loss: 0.2719 - accuracy: 0.8994 - val_loss: 0.2534 - val_accuracy: 0.9060
Epoch 7/20
56/56 - 0s - loss: 0.2654 - accuracy: 0.9003 - val_loss: 0.2494 - val_accuracy: 0.9083
Epoch 8/20
56/56 - 1s - loss: 0.2597 - accuracy: 0.9025 - val_loss: 0.2501 - val_accuracy: 0.9105
Epoch 9/20
56/56 - 1s - loss: 0.2572 - accuracy: 0.9030 - val_loss: 0.2475 - val_accuracy: 0.9038
Epoch 10/20
56/56 - 0s - loss: 0.25

Epoch 6/20
56/56 - 0s - loss: 0.3226 - accuracy: 0.8846 - val_loss: 0.2981 - val_accuracy: 0.8837
Epoch 7/20
56/56 - 1s - loss: 0.3177 - accuracy: 0.8857 - val_loss: 0.2934 - val_accuracy: 0.8859
Epoch 8/20
56/56 - 1s - loss: 0.3121 - accuracy: 0.8857 - val_loss: 0.2890 - val_accuracy: 0.8949
Epoch 9/20
56/56 - 1s - loss: 0.3093 - accuracy: 0.8882 - val_loss: 0.2812 - val_accuracy: 0.8949
Epoch 10/20
56/56 - 1s - loss: 0.3057 - accuracy: 0.8891 - val_loss: 0.2819 - val_accuracy: 0.8971
Epoch 11/20
56/56 - 1s - loss: 0.3012 - accuracy: 0.8888 - val_loss: 0.2769 - val_accuracy: 0.8971
Epoch 12/20
56/56 - 1s - loss: 0.3002 - accuracy: 0.8905 - val_loss: 0.2754 - val_accuracy: 0.8993
Epoch 13/20
56/56 - 1s - loss: 0.2956 - accuracy: 0.8924 - val_loss: 0.2813 - val_accuracy: 0.8904
Epoch 14/20
56/56 - 1s - loss: 0.2962 - accuracy: 0.8919 - val_loss: 0.2667 - val_accuracy: 0.9038
Epoch 15/20
56/56 - 1s - loss: 0.2945 - accuracy: 0.8882 - val_loss: 0.2653 - val_accuracy: 0.9060
Epoch 16/20
56

In [9]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10788), started 0:00:59 ago. (Use '!kill 10788' to kill it.)

In [16]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 8776), started 0:01:04 ago. (Use '!kill 8776' to kill it.)

## Test the model

In [10]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"saved_models\Model 1\Run-1")

In [11]:
test_pred = np.argmax(model.predict(test_inputs), axis=1)
test_accuracy = (test_targets == test_pred).sum() / test_targets.shape[0]

In [12]:
print('\nTest accuracy: {0:.2f}%'.format(test_accuracy * 100.))


Test accuracy: 91.29%
